In [1]:
import tensorflow as tf
import numpy as np
import tfutil

In [2]:
x = [[1, 2], [3, 4]]
const1 = tf.constant(np.array(x))
print(const1)
print(tf.shape(const1))
tfutil.print_constant(const1)

Tensor("Const:0", shape=(2, 2), dtype=int64)
Tensor("Shape:0", shape=(2,), dtype=int32)
[[1 2]
 [3 4]]


In [3]:
y = [1, 1]
row_to_add = tf.constant(np.array(y))
print(row_to_add)
print(tf.shape(row_to_add))
tfutil.print_constant(row_to_add)

Tensor("Const_1:0", shape=(2,), dtype=int64)
Tensor("Shape_1:0", shape=(1,), dtype=int32)
[1 1]


In [4]:
original_row = const1[0]
print(original_row)
print(tf.shape(original_row))
tfutil.print_constant(original_row)

Tensor("strided_slice:0", shape=(2,), dtype=int64)
Tensor("Shape_2:0", shape=(1,), dtype=int32)
[1 2]


In [5]:
updated_row = original_row + row_to_add
print(updated_row)
print(tf.shape(updated_row))
tfutil.print_operation_value(updated_row)

Tensor("add:0", shape=(2,), dtype=int64)
Tensor("Shape_3:0", shape=(1,), dtype=int32)
[2 3]


In [6]:
unchanged_indices = tf.range(tf.size(const1))
print(unchanged_indices)
print(tf.shape(unchanged_indices))
tfutil.print_operation_value(unchanged_indices)

Tensor("range:0", shape=(4,), dtype=int32)
Tensor("Shape_4:0", shape=(1,), dtype=int32)
[0 1 2 3]


In [7]:
changed_indices = tf.range(const1.get_shape()[0])
print(changed_indices)
print(tf.shape(changed_indices))
tfutil.print_operation_value(changed_indices)

Tensor("range_1:0", shape=(2,), dtype=int32)
Tensor("Shape_5:0", shape=(1,), dtype=int32)
[0 1]


In [8]:
a_flat = tf.reshape(const1, [-1])
print(a_flat)
print(tf.shape(a_flat))
tfutil.print_operation_value(a_flat)

Tensor("Reshape:0", shape=(4,), dtype=int64)
Tensor("Shape_6:0", shape=(1,), dtype=int32)
[1 2 3 4]


In [9]:
updated_a_flat = tf.dynamic_stitch([unchanged_indices, changed_indices], [a_flat, updated_row])
print(updated_a_flat)
print(tf.shape(updated_a_flat))
tfutil.print_operation_value(updated_a_flat)

Tensor("DynamicStitch:0", shape=(4,), dtype=int64)
Tensor("Shape_7:0", shape=(1,), dtype=int32)
[2 3 3 4]


In [10]:
updated_a = tf.reshape(updated_a_flat, const1.get_shape())
print(updated_a)
print(tf.shape(updated_a))
tfutil.print_operation_value(updated_a)

Tensor("Reshape_1:0", shape=(2, 2), dtype=int64)
Tensor("Shape_8:0", shape=(2,), dtype=int32)
[[2 3]
 [3 4]]


In [11]:
# Apply function (increments x_i) on elements for which a certain condition
# apply (x_i != -1 in this example).
x = [0.1, -1., 5.2, 4.3, -1., 7.4]
const1 = tf.constant(np.array(x))
print(const1)
print(tf.shape(const1))
tfutil.print_constant(const1)

Tensor("Const_2:0", shape=(6,), dtype=float64)
Tensor("Shape_9:0", shape=(1,), dtype=int32)
[ 0.1 -1.   5.2  4.3 -1.   7.4]


In [12]:
condition_mask=tf.not_equal(x,tf.constant(-1.))
print(condition_mask)
print(tf.shape(condition_mask))
tfutil.print_constant(condition_mask)

Tensor("NotEqual:0", shape=(6,), dtype=bool)
Tensor("Shape_10:0", shape=(1,), dtype=int32)
[ True False  True  True False  True]


In [13]:
partitioned_data = tf.dynamic_partition(x,
    tf.cast(condition_mask, tf.int32) , 2)
print(partitioned_data)
print(tf.shape(partitioned_data))

[<tf.Tensor 'DynamicPartition:0' shape=(?,) dtype=float32>, <tf.Tensor 'DynamicPartition:1' shape=(?,) dtype=float32>]
Tensor("Shape_11:0", shape=(2,), dtype=int32)


In [14]:
partitioned_data[1] = partitioned_data[1] + 1.0
print(partitioned_data[1])
print(tf.shape(partitioned_data[1]))
tfutil.print_constant(partitioned_data[1])

Tensor("add_1:0", shape=(?,), dtype=float32)
Tensor("Shape_12:0", shape=(1,), dtype=int32)
[ 1.10000002  6.19999981  5.30000019  8.39999962]


In [15]:
condition_indices = tf.dynamic_partition(
    tf.range(tf.shape(x)[0]), tf.cast(condition_mask, tf.int32) , 2)
print(condition_indices)
print(tf.shape(condition_indices))

[<tf.Tensor 'DynamicPartition_1:0' shape=(?,) dtype=int32>, <tf.Tensor 'DynamicPartition_1:1' shape=(?,) dtype=int32>]
Tensor("Shape_14:0", shape=(2,), dtype=int32)


In [16]:
ds_const1 = tf.dynamic_stitch(condition_indices, partitioned_data)
print(ds_const1)
print(tf.shape(ds_const1))
tfutil.print_constant(ds_const1)

Tensor("DynamicStitch_1:0", shape=(?,), dtype=float32)
Tensor("Shape_15:0", shape=(1,), dtype=int32)
[ 1.10000002 -1.          6.19999981  5.30000019 -1.          8.39999962]
